# 스레드를 블로킹 I/O용으로 사용하고 병렬화용으로는 사용하지 말자

Python의 표준 구현을 CPython이라고 한다. CPython은 파이썬 프로그램을 두 단계로 실행한다. 먼저 소스 텍스트를 `bytecode`로 파싱하고 컴파일한다. 이후 스택 기반 인터프리터로 bytecode를 실행한다. bytecode 인터프리터는 Python 프로그램이 실행되는 동안 지속되고, 일관성 있는 상태를 유지한다. Python은 전역 인터프리터 잠금(`GIL`, `Global Interpreter Lock`)이라는 메커니즘으로 일관성을 유지한다.

GIL은 상호 배제 잠금(mutex)이며 CPython이 선점형 멀티스레딩의 영향을 받지 않게 막아준다. 선점형 멀티스레딩은 한 스레드가 다른 스레드를 인터럽트해서 프로그램의 제어를 얻는 것을 말한다. 이 인터럽트가 예상치 못한 시간에 일어나면 인터프리터 상태가 망가진다. GIL은 이런 인터럽트를 막아주며 모든 bytecode 명령어가 CPython 구현과 C 확장 모듈에서 올바르게 동작함을 보장한다.

GIL은 중요한 부작용을 갖고 있다. C++나 자바 같은 언어로 작성한 프로그램에서 여러 스레드를 실행한다는 건 프로그램이 동시에 여러 CPU 코어를 사용함을 의미한다. Python도 멀티스레드를 지원하지만, GIL은 한 번에 한 스레드만 실행하게 한다. 다시 말해 스레드가 병렬 연산을 해야 하거나 파이썬 프로그램의 속도를 높여야 하는 상화아이라면 실망하게 될 것이다.

In [1]:
def factorize(number):
    for i in range(1, number + 1):
        if number % i == 0:
            yield i

In [3]:
from time import time

numbers = [2139079, 1214759, 1516637, 1852285]
start = time()
for number in numbers:
    list(factorize(number))
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.413 seconds


멀티 스레드로 다음 작업을 정의한다.

In [4]:
from threading import Thread

class FactoizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number
        
    def run(self):
        self.factors = list(factorize(self.number))

다음은 각 숫자를 인수 분해할 스레드를 병렬로 시작한다.

In [7]:
start = time()
threads = []
for number in numbers:
    thread = FactoizeThread(number)
    thread.start()
    threads.append(thread)

스레드가 작업을 마무리하기까지 기다린다.

In [8]:
for thread in threads:
    thread.join()
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.654 seconds


순서대로 처리했을 때 보다 시간이 더 많이 걸림을 알 수 있다. 이를 통해 GIL이 표준 CPython interpreter에서 실행하는 프로그램에 미치는 영향을 알 수 있다.

CPython이 멀티코어를 활용하게 하는 방법을 여러 가지지만, 표준 Thread 클래스에는 동작하지 않으므로 노력이 필요하다.  

## 파이썬이 스레드를 지원하는 이유는 두가지 있다.

### 1. 멀티스레드를 이용하면 프로그램이 동시에 여러 작업을 하는 것처럼 보이게 만들기 용이하다.

동시에 동작하는 태스크를 관리하는 코드는 구현하기 어렵다. 스레드를 통해 함수를 마치 병렬ㄹ로 실행하는 것처럼 해주는 일을 파이썬에 맡길 수 있다. 비록 GIL 떄문에 한 번에 한 스레드만 진행하지만, CPython은 파이썬 스레드가 어느 정도 공평하게 실행됨을 보장하기 때문이다.

### 2. 특정 유형의 시스템 호출을 수행할 때 일어나는 블로킹 I/O를 다루기 위해서다.

시스템 호출은 파이썬 프로그램에서 외부 환경과 대신 상호 작용하도록 컴퓨터 운영체제에 요청하는 방법이다. 블로킹 I/O로는 파일 읽기/쓰기, 네트워크와의 상호작용, 디스플레이 같은 장치와의 통신 등이 잇다. 스레드는 운영체제가 이런 요청에 응답하는 데 드는 시간을 프로그램과 분리하므로 블로킹 I/O를 처리할 때 유용하다.

예를 들면 원격 제어가 가능한 헬리콥터에 직렬 포트로 신호를 보내고 싶다 가정하자. 예제에서는 이 작업을 느린 시스템 호출(select)에 위임한다. 이 함수는 동기식 직렬 포트를 사용할 때 일어나는 상황과 비슷하게 하려고 운영체제에 0.1초간 블록한 후 제어를 프로그램에 돌려달라고 요청한다.

In [20]:
import select

def slow_systemcall():
    select.select([], [], [], 0.1)

이 시스템 호출을 연속해서 실행하면 시간이 선형으로 증가한다.

In [36]:
start = time()
for _ in range(5):
    slow_systemcall()
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.522 seconds


문제는 `slow_systemcall` 함수가 실행 동안 프로그램이 다른 일을 할 수 없다는 점이다. 프로그램의 메인 스레드는 시스템 호출 select 때문에 실행이 막혀있다. 신호를 보내는 동안 헬리콥터의 다음 이동을 계산해야 한다. 블로킹 I/O를 사용하면서 동시에 연산도 해야 한다면 시스템 호출을 스레드로 옮기는 방안을 고려해야 한다.

다음 코드는 `slow_systemcall` 함수를 별도의 스레드에서 여러 번 호출하여 실행한다. 이렇게 하면 동시에 여러 직렬 포트와 통신할 수 있게 되고, 메인 스레드는 필요한 계산이 무엇이든 수행하도록 남겨둘 수 있다.

In [35]:
start = time()
threads = []
for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)
    
def compute_helicopter_location(index):
    index += 1
    
for i in range(5):
    compute_helicopter_location(i)
for thread in threads:
    thread.join()
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.107 seconds


스레드가 시작하면 시스템 호출 스레드가 종료할 때까지 기다리기 전에 헬리콥터의 다음 이동을 계산한다.

병렬 처리 시간은 직렬 처리 시간보다 5배 짧다. 시스템 호출이 GIL의 제약을 받지만 여러 파이썬 스레드를 모두 병렬로 실행할 수 있음을 보여준다. GIL은 파이썬 코드가 병렬로 실행하지 못하게 한다. 하지만 시스템 호출에서는 이런 부정적인 영향이 없다. 이는 파이썬 스레드가 시스템 호출을 만들기 전에 GIL을 풀고 시스템 호출의 작업이 끝나는 대로 GIL을 다시 얻기 때문이다.

스레드 이외에도 내장 모듈 `asyncio`처럼 블로킹 I/O를 다루는 다양한 수단이 있고, 이런 대체 수단에는 중요한 이점이 있다. 하지만 이런 옵션을 선택하면 실행 모델에 맞춰서 코드를 재작성해야 하는 추가 작업이 필요하다. 스레드를 이용하면 프로그램의 수정을 최소화하면서 블로킹 I/O를 병렬로 수행하는 가장 간단한 방법이다.